In [1]:
import torch
import random
from diffusers import FluxPipeline

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


Demistifying Flux Architecture: https://arxiv.org/html/2507.09595v1

# High VRAM (A100)

In [ ]:
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)
pipe.enable_model_cpu_offload() #save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power

Keyword arguments {'dtype': torch.bfloat16} are not expected by FluxPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
prompts = ["a small cool canned cocacola bottle ","""A young, stylish Korean man with dark hair, wearing a black leather jacket over a white collared shirt and black tie.
 He has an open-mouthed expression and is raising his left hand in a waving or stopping gesture.
  the photo appears zoomed out. He stands outdoors in a studio with blurred neon lights and signs, suggesting an event or public appearance.
   Candid, dynamic shot, street photography"""]
extra_prompt = "low quality.keep the output real, compressed."
random_seed = random.randint(0, 2**32 - 1)
for i, prompt in enumerate(prompts):
  image = pipe(
      prompt,
      height=1024,
      width=1024,
      guidance_scale=2,
      num_inference_steps=50,
      max_sequence_length=512,
      generator=torch.Generator("cpu").manual_seed(random_seed)
  ).images[0]
  image.save(f"flux-image - {i}.png")

  0%|          | 0/50 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (82 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['dynamic shot, street photography']


  0%|          | 0/50 [00:00<?, ?it/s]

# Low VRAM (L4 GPU)

In [2]:
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", dtype=torch.bfloat16)

model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/273 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/19.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [3]:
pipe.enable_sequential_cpu_offload() # extreme VRAM saving, slow inference

In [4]:
prompts = ["Four miniature Coca-Cola cans (Classic, Light, Diet Coke, Zero Sugar) lined up on a light wooden surface. Photorealistic, close-up, sharp focus.",
           """A young, stylish Korean man with dark hair, wearing a black leather jacket over a white collared shirt and black tie.
              He has an open-mouthed expression and is raising his left hand in a waving gesture.
              the photo appears zoomed out. He stands outdoors iwith blurred neon lights and signs,
              low quality.keep the output real, compressed.""",
           """A young, stylish Korean man with dark hair, wearing a black leather jacket over a white collared shirt and black tie.
              He has an open-mouthed expression and is raising his left hand in a waving gesture.
              the photo appears zoomed out. He stands outdoors iwith blurred neon lights and signs."""
           ]

In [5]:
random_seed = random.randint(0, 2**32 - 1)

for i, prompt in enumerate(prompts):
  image = pipe (
      prompt = prompt,
      height = 512,
      width = 768,
      guidance_scale = 2,
      num_inference_steps = 50,
      max_sequence_length = 512,
      generator=torch.Generator("cpu").manual_seed(random_seed)
  ).images[0]
  image.save(f"flux-image - {i}.png")


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]